In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from tqdm import tqdm
import json
from anytree.importer import JsonImporter
from anytree import PreOrderIter
import pandas as pd
import sys
sys.path.append("utils/")
from utils.TreePlotter import TreePlotter
from utils.TreeConverter import TreeConverter
import math
# from torchtext.data import Field, BucketIterator

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = 'cpu'

In [224]:
def prepare_data():
    reserved_tokens_path = '../data/ast_trees/reserved_tokens.json'
    asts_path = '../data/ast_trees/asts.csv.bz2'

    with open(reserved_tokens_path, 'r') as json_f:
        json_data = json_f.read()

    reserved_tokens = json.loads(json_data)

    asts = pd.read_csv(asts_path, chunksize=1e4)

    importer = JsonImporter()
    
#     reserved_tokens['<sos>'] = len(reserved_tokens)
#     reserved_tokens['<eos>'] = len(reserved_tokens)
    reserved_tokens['<pad>'] = len(reserved_tokens)


    data = []
    try:
        for ast_chunk in asts:
            for ast in ast_chunk['AST']:
                tree = importer.import_(ast)
                tree_repr = torch.tensor([node.token for node in PreOrderIter(tree) if node.res])
                if len(tree_repr) < 300 and len(tree_repr) > 1:
                    data.append(tree_repr)
    except Exception:
        pass
    
    # Sort trees such that during batching we have to pad the batches minimally
    data.sort(key=len)

#     print(torch.stack(trees))
#     data = torch.tensor(trees)
    
    return data, reserved_tokens

data, reserved_tokens = prepare_data()    

In [13]:
def preprocess_batch(batch):
#     batch = [torch.tensor([reserved_tokens['<sos>']] + item + [reserved_tokens['<eos>']]) for item in batch]
    batch = nn.utils.rnn.pad_sequence(batch, batch_first=True, padding_value=reserved_tokens['<pad>'])
    
    return batch.to(device)


In [329]:
class EncoderVAE(nn.Module):
    def __init__(self, vocab_size, embedding_size, hidden_size, latent_size, dropout, bidirectional=False):
        super().__init__()       
        
        # Variables
        self.latent_size = latent_size
        self.bidirectional = bidirectional
        self.hidden_size = hidden_size

        # Layers
        self.embedding = nn.Embedding(vocab_size, embedding_size, padding_idx=reserved_tokens['<pad>'])
        self.dropout = nn.Dropout(dropout)
        self.lstm = nn.LSTM(embedding_size, hidden_size, bidirectional=self.bidirectional, batch_first=True)
        self.z_mean = nn.Linear(hidden_size, self.latent_size)
        self.z_log_var = nn.Linear(hidden_size, self.latent_size)
        
        
    def reparameterize(self, mu, log_var):
        if self.training:
            std = torch.exp(0.5 * log_var)
            eps = std.data.new(std.size()).normal_()
            return eps.mul(std).add_(mu)
        else:
            return mu
        
    def forward(self, inp):
        # inp shape: (batch_size, seq_len)
        
        [batch_size, seq_len] = inp.size()
        
        embedding = self.dropout(self.embedding(inp))
        
        # Get final hidden state from the last layer
        outputs, (hidden, cell) = self.lstm(embedding)
        
        outputs = outputs[:, -1, :].view(1, batch_size, -1)
#         print(outputs.shape)
#         print(hidden.shape)
#         print(cell.shape)
        
#         print(cell.view(1, batch_size, self.hidden_size).shape)
        
#         last_hidden_state = hidden[:, -1]
        
#         z_mean = self.z_mean(last_hidden_state)
#         z_log_var = self.z_log_var(last_hidden_state)

        z_mean = self.z_mean(outputs)
        z_log_var = self.z_log_var(outputs)
        
        z = self.reparameterize(z_mean, z_log_var)
        
        return z, z_mean, z_log_var

Epoch: 9: 100%|██████████| 213/213 [00:55<00:00,  3.81batch/s, loss=2.93]


In [394]:
class DecoderVAE(nn.Module):
    def __init__(self, latent_size, hidden_size, output_size):
        super().__init__()
        
        self.latent_size = latent_size
        self.hidden_size = hidden_size
        
        self.lstm = nn.LSTM(latent_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=2)
        
    def forward(self, z, seq_len, target=None):
        # z shape: (1, batch_size, latent_variable_size)
        
        [_, batch_size, latent_variable_size] = z.size()
        
        # Replicate z to append the same z at each time step
        z = torch.cat([z] * seq_len, 1).view(batch_size, seq_len, self.latent_size) 

#         z.squeeze(0)

        # If we are not training
#         if target is None:

#             outputs = torch.zeros(seq_len, batch_size, self.hidden_size).to(device)

#             output, (hidden, cell) = self.lstm(z)
#             # shape of outputs: (batch_size, seq_len, hidden_size)

#             outputs[0] = output

#             for i in range(1, seq_len):
#                 output, (hidden, cell) = self.lstm(output, (hidden, cell))
#                 outputs[i] = output

#             outputs = outputs.view(batch_size, seq_len, -1)
            
#         else:
#             z = torch.cat([z] * seq_len, 1).view(batch_size, seq_len, self.latent_size)
#             print(z.shape)
#             print(target.shape)
#             decoder_inp = torch.cat([target, z], 2)
            
#             print(decoder_inp.shape)

        outputs, _ = self.lstm(z)


        out = self.softmax(self.fc(outputs))
        # shape of out: (batch_size, seq_len, output_size)
        
        return out
        
            

In [395]:
class VAE(nn.Module):
    def __init__(self, encoder, decoder):
        super().__init__()
#         self.embedding = nn.Embedding(input_size, embedding_size)
#         self.dropout = nn.Dropout(dropout)
        self.encoder = encoder
        self.decoder = decoder
            
    def forward(self, inp):
        [batch_size, seq_len] = inp.size()
        
#         embedded = self.dropout(self.embedding(inp))
        # embedded shape: (batch_size, seq_length, embedding_size)
        
        z, z_mean, z_log_var = self.encoder(inp)
        dec_output = self.decoder(z, inp.size()[-1], inp)
        
        return dec_output, z_mean, z_log_var
        
            
    def loss(self, dec_output, target, mu, log_var):
        # Calculate latent loss/reconstruction loss
        # How far is the reconstruction from the original input
        # reconstruction shape: (batch_size, seq_len, vocab_len) -> (batch_size * seq_len, vocab_len)
        # target shape: (batch_size, seq_len) -> (batch_size * seq_len)
        reconstruction_loss = F.nll_loss(
            m(dec_output).view(-1, len(reserved_tokens)),
            target.view(-1),
            ignore_index=reserved_tokens['<pad>']
        )
        
        # Get kullback leichner loss
        kl_loss = 0.5 * torch.sum(log_var.exp() - log_var - 1 + mu.pow(2))
        
#         print(reconstruction_loss, kl_loss)
        
        return reconstruction_loss + kl_loss
    
    def train(self, data, epochs, batch_size):
        self.encoder.train()
        self.decoder.train()
        
        encoder_optimizer = optim.Adam(self.encoder.parameters(), lr=0.001)
        decoder_optimizer = optim.Adam(self.decoder.parameters(), lr=0.001)
        
        
        for epoch in range(epochs):
            pbar = tqdm(range(math.ceil(len(data)/batch_size)), unit='batch', desc=f'Epoch: {epoch}', position=0)
            for i in range(0, len(data), batch_size):
                encoder_optimizer.zero_grad()
                decoder_optimizer.zero_grad()

                batch = preprocess_batch(data[i: i + batch_size])

                dec_output, z_mean, z_log_var = self(batch)
                loss = self.loss(dec_output, batch, z_mean, z_log_var)
                loss.backward()
                encoder_optimizer.step()
                decoder_optimizer.step()
                pbar.set_postfix(loss=round(loss.item(), 3))
                pbar.update()
            print(f'Epoch {epoch} loss: {loss}')
        

Epoch: 0:   0%|          | 0/213 [01:37<?, ?batch/s]


In [396]:
vocab_size = len(reserved_tokens)
embed_dim = 300
lstm_hidden_size = 256
latent_size = 128
dropout = 0.5

encoder = EncoderVAE(vocab_size, embed_dim, lstm_hidden_size, latent_size, dropout).to(device)
decoder = DecoderVAE(latent_size, lstm_hidden_size, vocab_size).to(device)

vae = VAE(encoder, decoder)





In [397]:
vae.train(data, 3, 64)

Epoch: 0:  43%|████▎     | 91/213 [00:05<00:10, 11.98batch/s, loss=3.54]

KeyboardInterrupt: 

Epoch: 0:  43%|████▎     | 92/213 [00:25<00:10, 11.98batch/s, loss=3.54]

In [379]:
idx = 4

encoder.eval()
decoder.eval()

z, _, _ = encoder(data[idx].view(1, -1).to(device))
reconstruction = torch.argmax(decoder(z, len(data[idx])), dim=2)

print(data[idx])
print(reconstruction)

tensor([ 0, 11,  8, 12, 13, 18,  5,  6,  7,  8])
tensor([[7, 8, 8, 8, 8, 8, 8, 8, 8, 8]], device='cuda:0')


In [353]:
len(data[idx])

35

In [281]:
decoder(z, len(data[-1])).shape

Epoch: 9: 100%|██████████| 213/213 [01:04<00:00,  3.30batch/s, loss=2.97]


torch.Size([1, 299, 136])

In [290]:
reserved_tokens

{'root': 0,
 'TYPEDEF_DECL': 1,
 'TYPE_DEF': 2,
 'IDENTIFIER': 3,
 'NAMESPACE_REF': 4,
 'VAR_DECL': 5,
 'TYPE': 6,
 'DECLARATOR': 7,
 'NAME': 8,
 'INTEGER_LITERAL': 9,
 'UNARY_OPERATOR_-': 10,
 'FUNCTION_DECL': 11,
 'RETURN_TYPE': 12,
 'COMPOUND_STMT': 13,
 'CALL_EXPR': 14,
 'ARGUMENTS': 15,
 'TYPE_REF': 16,
 'DECL_REF_EXPR': 17,
 'DECL_STMT': 18,
 'BINARY_OPERATOR_+': 19,
 'FOR_STMT': 20,
 'BINARY_OPERATOR_=': 21,
 'BINARY_OPERATOR_<': 22,
 'UNARY_OPERATOR_POST_++': 23,
 'ARRAY_SUBSCRIPT_EXPR': 24,
 'BINARY_OPERATOR_!=': 25,
 'CHARACTER_LITERAL': 26,
 'NULL_STMT': 27,
 'STRING_LITERAL': 28,
 'WHILE_STMT': 29,
 'UNARY_OPERATOR_!': 30,
 'MEMBER_REF_EXPR': 31,
 'BINARY_OPERATOR_%': 32,
 'PAREN_EXPR': 33,
 'BINARY_OPERATOR_*': 34,
 'IF_STMT': 35,
 'BINARY_OPERATOR_&&': 36,
 'BINARY_OPERATOR_<=': 37,
 'CSTYLE_CAST_EXPR': 38,
 'CXX_FOR_RANGE_STMT': 39,
 'PARM_DECL': 40,
 'RETURN_STMT': 41,
 'BINARY_OPERATOR_==': 42,
 'UNARY_OPERATOR_&': 43,
 'FLOATING_LITERAL': 44,
 'STRUCT_DECL': 45,
 'FIE

In [12]:
TreePlotter.plot_tree(root, 'tree.png')

In [13]:
binary_tree = TreeConverter.to_binary(root)

In [14]:
TreePlotter.plot_tree(binary_tree, 'binary_tree.png', binary=True)

In [18]:
tree_repr = torch.tensor([node.token for node in PreOrderIter(root) if node.res])

In [18]:
root.children[0].children[1].children[0].children[0]

AnyNode(res=False, token='')